# Chapter 3 Linear regression Exercise

In this exercise you will learn more about feature columns, work with estimators other than LinearRegressor, experiment with hyperparameters.

You will need to complete the notebook by filling in missing script that is marked by **'... # TODO n'** .


## Feature Columns

### Exercise 1. 
In this exercise you will add two feature columns to LinearRegressor model

In [ ]:
# import libraries
import tensorflow as tf
''' "fc.numeric_column" represents real valued or numerical features;
    Reference - https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column'''
from tensorflow.feature_column import numeric_column
'''A CategoricalColumn with in-memory vocabulary. Use this when your inputs are in string or integer format, and you have 
    an in-memory vocabulary mapping each value to an integer ID. By default, out-of-vocabulary values are ignored. Use either
    (but not both) of num_oov_buckets and default_value to specify how to include out-of-vocabulary values.
    Reference - https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list?hl=ko'''
from tensorflow.feature_column import categorical_column_with_vocabulary_list
''' Bucketized column 
    Reference - https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column?hl=ko '''
from tensorflow.feature_column import bucketized_column

#### Define the feature columns that will be used to train the regressor. Our dataset consists of 'area', a numeric value signifying the area of house,  'type', if a house is bunglow or apartment, 'age' numeric column representing house age in years and 'age_buckets' column

In [ ]:
# define age feature column
age = numeric_column("age")
# bucketizing enables the model to clearly distinguish between different year categories
# define age bucket column by providing reference to source column and defining boundaries.
age_buckets = bucketized_column(age, [0, 20, 40, 60, 80, 100] ) # TODO 1: add age bucket boundaries at 0, 20, 40, 60, 80, 100 years
featcols = [
numeric_column("area"),
categorical_column_with_vocabulary_list("type",["bungalow","apartment"]),
age,age_buckets # TODO 2: add age_buckets column
]

### Exercise 2.
Defining input function for training the model. This function returns a tuple containing features and labels data

In [ ]:
def train_input_fn():
    features = {"area":[1000,2000,4000,1000,2000,4000],
                # add rooms feature
                "rooms": [2, 3, 5, 2, 4, 4],
                # add age feature
                "age": [0, 20, 40, 60, 80, 100], # TODO 3: add age feature training data 
                "type":["bungalow","bungalow","house",
                      "apartment","apartment","apartment"]}
    labels = [ 500 , 1200 , 1500 , 700 , 1300 , 1900 ]
    return features, labels

### Defining input function for model predictions. 

In [ ]:
# We define a test data function to be passed to model for predictions. This function returns features
def predict_input_fn():
    features = {"area":[1500,1800],
                "rooms":[2,3],
              "type":["house","apartment"],
              "age":[21,8] # TODO 4: add "age" column test data 
              }
    return features

# LinearRegressor estimator model

### Instantiate LinearRegressor model

In [ ]:
model = tf.estimator.LinearRegressor(featcols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp844me08g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Train and fit LinearRegressor model to input dataset

In [ ]:
model.train(train_input_fn, steps=200)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp844me08g/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1621666.6, step = 0
INFO:tensorflow:global_step/sec: 891.18
INFO:tensorflow:loss = 50820.88, step = 100 (0.114 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 200...
INFO:tensorflow:Saving checkpoints for 200 into /tmp/tmp844me08g/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 200...
INFO:tensorflow:Loss for final step: 44585.66.


### LinearRegressor Prediction. 

In [ ]:
# next() method is used as an iterator. It returns the next input line
predictions = model.predict(predict_input_fn)
print(next(predictions))
print(next(predictions))

INFO:tensorflow:Calling model_fn.


Exception ignored in: <generator object Estimator.predict at 0x7fe4fec07ca8>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 641, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5588, in get_controller
    yield g
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5400, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp844me08g/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([694.17773], dtype=float32)}
{'predictions': array([797.7711], dtype=float32)}


# Boosted Tree Regressor model



### Instantiate Boosted Tree Regressor model

In [ ]:
# data will fit in memory , so we can use 1 batch
n_batches = 1
# instantiate BoostedTreesRegressor model
btr_model = tf.estimator.BoostedTreesRegressor(featcols, n_batches_per_layer=n_batches)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpx0zq1ut4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Exercise 3.
Train and fit BoostedTreeRegressor model to input dataset

In [ ]:
# train Boosted Trees Regressor model on train data provided by train_input_fn
btr_model.train(train_input_fn, steps=200)  # TODO 5: add code to train btr_model on train data with 200 steps

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpx0zq1ut4/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners after saving checkp

### Exercide 4.
BoostedTreesRegressor Prediction

In [ ]:
# generate a prediction using predict data
btr_predictions = btr_model.predict(predict_input_fn) # TODO 6: add code to generate pridiction using predict_input_fn
print(next(btr_predictions))
print(next(btr_predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpx0zq1ut4/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([1304.5244], dtype=float32)}
{'predictions': array([1200.0005], dtype=float32)}


Exception ignored in: <generator object Estimator.predict at 0x7fe4fea0a6d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 641, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5588, in get_controller
    yield g
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5400, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


# Boston House Predictions

### In this section we will use BoostedTreesRegressor to predict house prices using Boston dataset and compare results with Linear Refressor. We will also play with hyperparameters in order to reduce loss function

In [ ]:
# import libraries
import pandas as pd
'''Import keras API for readily available dataset.

Reference - https://www.tensorflow.org/api_docs/python/tf/keras/datasets'''
from tensorflow.keras.datasets import boston_housing

In [ ]:
# load train and test data
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [ ]:
# define features and feature columns
features = ['CRIM', 'ZN', 'INDUS','CHAS','NOX','RM','AGE',
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()



,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [ ]:
# feature columns
feature_columns = []
for feature_name in features:
    ''' "fc.numeric_column" represents real valued or numerical features;
        Reference - https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column
        
    "dtype" defines the type of values.'''
    feature_columns.append(numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)


[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTRATIO', shape=(1

In [ ]:
def estimator_input_fn(df_data, df_label, epochs=20, shuffle=True, batch_size=64):
    def input_function():
        '''With the help of tf.data.Dataset.from_tensor_slices() method, we can get the slices of an array in the form of 
            objects by using tf.data.Dataset.from_tensor_slices() method.
            Reference - https://www.geeksforgeeks.org/tensorflow-tf-data-dataset-from_tensor_slices/ '''
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle: 
            '''shuffles the original list of number'''
            ds = ds.shuffle(100)
        '''Batch size - Batch size stands for the number of training examples in one forward/backward pass. The higher the
        batch size, the more memory space you'll need.
        
        Epochs - In neural networks, one epoch means one forward pass and one backward pass of all the training examples. 
        Example: if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete
        1 epoch.'''
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

'''function call assignment to object'''
train_input_fn = estimator_input_fn(x_train_df, y_train_df)
val_input_fn = estimator_input_fn(x_test_df, y_test_df, epochs=1, shuffle=False)

### Linear Regressor Estimator



In [ ]:
# Instantiate a LinearRegressor estimator, 
# we train it using training data using 'train_input_fn' function, 
# and find result for validation dataset by evaluating the trained model using 'val_input_fn' function. 
# This is the model training and validation phase.
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir = 'logs/func/')
linear_est.train(train_input_fn, steps=100)
result = linear_est.evaluate(val_input_fn)
print(pd.Series(result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs/func/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in 

Exception ignored in: <generator object Estimator.predict at 0x7fe4facf34c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 641, in predict
    for key, value in six.iteritems(preds_evaluated)
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5588, in get_controller
    yield g
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5400, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


#### Model Prediction 

In [ ]:
result = linear_est.predict(val_input_fn)
for pred,exp in zip(result, y_test[:32]):
    print("Predicted Value: ", pred['predictions'][0], "Expected: ", exp)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/func/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted Value:  3.6583881 Expected:  7.2
Predicted Value:  23.55529 Expected:  18.8
Predicted Value:  21.86346 Expected:  19.0
Predicted Value:  24.590088 Expected:  27.0
Predicted Value:  22.883322 Expected:  22.2
Predicted Value:  21.195955 Exp

### Exercise 5.
BoostedTreesRegressor


#### 
Instantiate BoostedTreesRegressor model

In [ ]:
n_batches_per_layer=1
# Instantiate BoostedTreesRegressor model on Boston feature columns and using 1 batch per layer
boosted_trees_est = tf.estimator.BoostedTreesRegressor(feature_columns, n_batches_per_layer=n_batches_per_layer) #TODO 7: add script to instantiate BoostedTreesRegressor estimator


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpefx2e0fk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### Train and fit BoostedTreesRegressor to train dataset

In [ ]:
# train boosted_trees_est using train_input_fn with 100 max_steps
boosted_trees_est.train(train_input_fn, steps=100) #TODO 8: add script to train boosted_trees_est model


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpefx2e0fk/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners after saving checkp

#### Evaluation using validation dataset

In [ ]:
# use boosted_trees_est trained model to evaluate the result on validation data
eval_result = boosted_trees_est.evaluate(val_input_fn) #TODO 9: add script to evaluate the boosted_trees_est model on the validation data returned by val_input_fn function
print(pd.Series(eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-11-17T17:13:39Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpefx2e0fk/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20716s
INFO:tensorflow:Finished evaluation at 2020-11-17-17:13:39
INFO:tensorflow:Saving dict for global step 100: average_loss = 16.879652, global_step = 100, label/mean = 23.078432, loss = 18.828531, prediction/mean = 22.799252
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmpefx2e0fk/model.ckpt-100
average_loss        16.879652
label/mean          23.078432
loss                18.828531
prediction/mean     22.799252
glo

#### Boosted Trees Regressor prediction

In [ ]:
# Predict house prices 
boosted_trees_result = boosted_trees_est.predict(val_input_fn) #TODO 10: add script to predict house prices using trained boosted trees estimator on validation data that is returned from  val_input_fn function
# Print out predicted and expected results
for pred,exp in zip(boosted_trees_result, y_test[:32]):
    print("Predicted Value: ", pred['predictions'][0], "Expected: ", exp)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpefx2e0fk/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted Value:  10.356474 Expected:  7.2
Predicted Value:  18.051085 Expected:  18.8
Predicted Value:  20.79734 Expected:  19.0
Predicted Value:  34.395912 Expected:  27.0
Predicted Value:  22.042578 Expected:  22.2
Predicted Value:  20.988823 Expected:  24.5
Predicted Value:  29.376316 Expected:  31.2
Predicted Value:  22.295341 Expected:  22.9
Predicted Value:  21.949217 Expected:  20.5
Predicted Value:  17.953133 Expected:  23.2
Predicted Value:  21.512226 Expected:  18.6
Predicted Value:  19.028713 Expected:  14.5
Predicted Value:  15.534652 Expected:  17.8
Predicted Value:  41.87204 Expected:  50.0
Predicted Value:  16.048609 Expected:  20.8
Predicted Value:  21.897575 Expected:  24.3
Predicted Value:  22.204304 Expe

### Boosted Trees with hyperparameter tuning

In [ ]:
# define parameters
params = {
  'n_trees': 5,
  'max_depth': 5,
  'n_batches_per_layer': 1  
}
# instantiate BoostedTreesRegressor
boosted_trees_est = tf.estimator.BoostedTreesRegressor(feature_columns, **params) #TODO 11: instantiate BoostedTreesRegressor with hyperparameters.

# train boosted_trees_est using train_input_fn with 100 max_steps
boosted_trees_est.train(train_input_fn, steps=100) #TODO 12 : add script to train boosted_trees_est model

# evaluate estimator
eval_result = boosted_trees_est.evaluate(val_input_fn) #TODO 13: add script to evaluate the boosted_trees_est model on the validation data returned by val_input_fn function
# print evaluation result
print(pd.Series(eval_result))

# TODO 14: Experiment with different values of hyperparameters.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpxold3f6p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [ ]:
# define parameters
params = {
  'n_trees': 10,
  'max_depth': 8,
  'n_batches_per_layer': 1  
}
# instantiate BoostedTreesRegressor
boosted_trees_est = tf.estimator.BoostedTreesRegressor(feature_columns, **params) #TODO 11: instantiate BoostedTreesRegressor with hyperparameters.

# train boosted_trees_est using train_input_fn with 100 max_steps
boosted_trees_est.train(train_input_fn, steps=100) #TODO 12 : add script to train boosted_trees_est model

# evaluate estimator
eval_result = boosted_trees_est.evaluate(val_input_fn) #TODO 13: add script to evaluate the boosted_trees_est model on the validation data returned by val_input_fn function
# print evaluation result
print(pd.Series(eval_result))

# TODO 14: Experiment with different values of hyperparameters.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpooh5rspp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [ ]:
# define parameters
params = {
  'n_trees': 2,
  'max_depth': 6,
  'n_batches_per_layer': 1  
}
# instantiate BoostedTreesRegressor
boosted_trees_est = tf.estimator.BoostedTreesRegressor(feature_columns, **params) #TODO 11: instantiate BoostedTreesRegressor with hyperparameters.

# train boosted_trees_est using train_input_fn with 100 max_steps
boosted_trees_est.train(train_input_fn, steps=100) #TODO 12 : add script to train boosted_trees_est model

# evaluate estimator
eval_result = boosted_trees_est.evaluate(val_input_fn) #TODO 13: add script to evaluate the boosted_trees_est model on the validation data returned by val_input_fn function
# print evaluation result
print(pd.Series(eval_result))

# TODO 14: Experiment with different values of hyperparameters.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpcjpc6cua', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [ ]:
# define parameters
params = {
  'n_trees': 20,
  'max_depth': 7,
  'n_batches_per_layer': 1  
}
# instantiate BoostedTreesRegressor
boosted_trees_est = tf.estimator.BoostedTreesRegressor(feature_columns, **params) #TODO 11: instantiate BoostedTreesRegressor with hyperparameters.

# train boosted_trees_est using train_input_fn with 100 max_steps
boosted_trees_est.train(train_input_fn, steps=100) #TODO 12 : add script to train boosted_trees_est model

# evaluate estimator
eval_result = boosted_trees_est.evaluate(val_input_fn) #TODO 13: add script to evaluate the boosted_trees_est model on the validation data returned by val_input_fn function
# print evaluation result
print(pd.Series(eval_result))

# TODO 14: Experiment with different values of hyperparameters.

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpac3kgj9v', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
